In [17]:
from textblob import TextBlob 
from datetime import datetime
from pythainlp import word_tokenize
from nltk.corpus import stopwords
import nltk
import langdetect
import tweepy as tw
import pandas as pd
import re
import glob
import os
import schedule
import time
import requests
import shutil
from pythainlp.corpus import thai_stopwords
import string

In [18]:
#tweet_data.csv to keyword files
import glob
import pandas as pd
start = 0
filenames = glob.glob(str(str(os.getcwd())+"\\Data\\*.csv"))
for file in filenames:
    df1 = pd.read_csv(file)
    if start != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        start += 1
keys = list(set(df['Keyword'].tolist()))

df["Time"] = pd.to_datetime(df["Time"]).dt.strftime('%Y-%m-%d')
keys = list(set(df['Keyword'].tolist()))
folder = "collectkeys"
if not os.path.exists(folder):
    os.mkdir(folder)    
for key in keys:
    path = str(folder+'/'+key)
    dff = df.loc[df['Keyword'].isin([key])]
    days = list(set(dff['Time'].tolist()))
    if not os.path.exists(path):
        os.mkdir(path)
    for d in days:
        dfff = dff.loc[dff['Time'].isin([d])]
        dfff.to_csv(path+'/'+key+'_'+d+'.csv',encoding='utf-8',index=False)
print('collect complete')

collect complete


In [26]:
class DataManager:
    def __init__(self):
        ##-------------------- twitter --------------------##
        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "kE7s0TJ00spb9kEPZ1BC7w8A16dpy8Cr"}
        consumer_key = "EaFU9nJw2utR0lo2PUmJE3VZy"
        consumer_secret = "DsZuVw0tEl6GHhyK08tunsOE9ICSfwplEhRDMQwB8VIqngZ6i8"
        access_token = "759317188863897600-nuwQmcYfDX8lvdRyw2eCD6fMRMkLzzZ"
        access_token_secret = 'zFFc5OJywNMBrRAblI7kFV62ZTZPHfTU1Q5kZ1cKzUupD'
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self._api = tw.API(auth, wait_on_rate_limit=True)
        self.keys = os.listdir("collectkeys")
        self.df = None
        self._start = 0
        self.filenames = []

    def getSentimentENG(self,text):
        if TextBlob(text).sentiment.polarity > 0:
            return 'positive'
        elif TextBlob(text).sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'

    def getSentimentTH(self,text):
        # print(text)
        text = re.sub(r'[%]',' ',text)
        # print(text)
        params = {'text':text}
        # print(json.dumps(params, sort_keys=False, indent=4))
        try:
            response = requests.get(self._url, headers=self._headers, params=params)
            try:
                polarity = str(response.json()['sentiment']['polarity'])
            except (KeyError):
                polarity = 'neutral'
        except requests.exceptions.JSONDecodeError:
            polarity = 'URI too long'
            pass
        return polarity

    def formatdatetime(self,column):
        self.df[column] = pd.to_datetime(self.df[column]).dt.strftime('%Y/%m/%d')
        self.df[column] = pd.to_datetime(self.df[column])
    
    def sortdf(self,columns):
        self.df.sort_values(by=columns,inplace=True)
        return self.df

    def unionfile(self,filenames):              #type filename -> list
        self._start = 0
        self.filenames = filenames
        for file in filenames:
            df1 = pd.read_csv(file)
            if self._start != 0:
                self.df = pd.concat([self.df,df1])
                self.df.drop_duplicates(keep='last',inplace=True)
            else:
                self.df = df1
                self._start += 1
        self.keys = list(set(self.df['Keyword'].tolist()))
        self.collectfile()
        return self.df
    
    def newUnion(self):
        path=os.getcwd()
        keys = []
        start = 0
        for f in glob.glob(path+'/collectkeys/*'):
            keyname = os.path.split(f)[-1]
            keys.append(keyname)
        for k in keys:
            for file in glob.glob(path+'/collectkeys/'+k+'/*.csv'):
                if start == 0:
                    self.df = pd.read_csv(file)
                    start +=1
                else:
                    dff = pd.read_csv(file)
                    self.df = pd.concat([self.df,dff])
        self.keys = keys
        #self.collectfile()
        return self.df
    
    def setnewdf(self,dataframe):
        self.df = dataframe
        return self.df
    
    def concatfile(self,dataframe):
        self.df = pd.concat([self.df,dataframe])
        self.df.drop_duplicates(keep='last',inplace=True)
        self.df.sort_values(by=['Keyword'],inplace=True)
        self.formatdatetime('Time')
        return self.df
    
    def setdefaultDF(self):
        #self.df = self.unionfile(self.filenames)
        self.df = self.newUnion()
        return self.df
    
    def collectfile(self):
        self.df["Time"] = pd.to_datetime(self.df["Time"]).dt.strftime('%Y-%m-%d')
        keys = list(set(self.df['Keyword'].tolist()))
        folder = "collectkeys"
        if not os.path.exists(folder):
            os.mkdir(folder)    
        for key in keys:
            path = str(folder+'/'+key)
            dff = self.df.loc[self.df['Keyword'].isin([key])]
            days = list(set(dff['Time'].tolist()))
            if not os.path.exists(path):
                os.mkdir(path)
            for d in days:
                dfff = dff.loc[dff['Time'].isin([d])]
                dfff.to_csv(path+'/'+key+'_'+d+'.csv',encoding='utf-8',index=False)
        print('collect complete')

    def getperiod(self,since,until):  ####column for twitter
        self.formatdatetime('Time')
        dff = self.df
        dff.sort_values(by=['Time','Keyword'],inplace=True)
        if since == None and until != None:
            mask = (dff['Time']<=until)
        elif since != None and until == None:
            mask = (dff['Time']>=since)
        elif since != None and until != None:
            mask = (dff['Time']>=since) & (dff['Time']<=until)
        else:
            return dff
        return dff.loc[mask]

    def getrowwithkeys(self,keys):              #type keys -> list
        df = self.df
        return df.loc[df['Keyword'].isin(keys)]

    def collectwords(self,dataframe):
        #print(dataframe)
        nltk.download('stopwords')          #important
        dataframe = dataframe.reset_index()
        th_stopwords = list(thai_stopwords())
        en_stops = set(stopwords.words('english'))
        en_stops.update(list(string.ascii_lowercase))
        en_stops.update(list(string.ascii_uppercase))
        word = {}
        for index,row in dataframe.iterrows():    #only tweet
            if row['Language'] == 'en':
                allwords = str(row['Tweet']).split()
                for w in allwords: 
                    if w not in en_stops:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
            elif row['Language'] == 'th':
                allwords = word_tokenize(row['Tweet'], engine='newmm')
                for w in allwords: 
                    if w not in th_stopwords:
                        if w in word:
                            word[w] += 1
                        else:
                            word[w] = 1
            else:
                pass
        if 'RT' in word:
            del word['RT']  #for twitter
        if ' ' in word:
            del word[' ']   #for thai language
        sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
        worddf = pd.DataFrame(sortword,columns=['Word','Count'])
        return worddf   #word dataframe
        #return sortword     #tuple in list
    
    def deletekeyword(self,keyword):
        path=os.getcwd()
        for k  in keyword:
            shutil.rmtree(path+'//collectkeys//'+k+'//')
            self.keys.remove(k)
            #self.df.drop(self.df[self.df['Keyword']==k].index,inplace = True)
            self.df = self.newUnion()
        return self.df

In [20]:
class Twitter_Scrap:
    def __init__(self):

        #twitter api
        consumer_key = "EaFU9nJw2utR0lo2PUmJE3VZy"
        consumer_secret = "DsZuVw0tEl6GHhyK08tunsOE9ICSfwplEhRDMQwB8VIqngZ6i8"
        access_token = "759317188863897600-nuwQmcYfDX8lvdRyw2eCD6fMRMkLzzZ"
        access_token_secret = 'zFFc5OJywNMBrRAblI7kFV62ZTZPHfTU1Q5kZ1cKzUupD'
        auth = tw.OAuthHandler(consumer_key, consumer_secret)
        auth.set_access_token(access_token, access_token_secret)
        self._api = tw.API(auth, wait_on_rate_limit=True)

        #thai api
        self._url = "https://api.aiforthai.in.th/ssense"                     
        self._headers = {'Apikey': "vIQAf35aRkc7QUbR1fTPvzvtkqtSKAaz"}

        self.df = None
        self.keys = []

    def setdataframe(self,df):
        self.df = df
        self.keys = self.df['Keyword'].tolist()
        self.keys = list(set(self.keys))

    def getheader(self):
        return self.df.columns.tolist()
    
    def getSentiment(self,text):

        if TextBlob(text).sentiment.polarity > 0:
            return 'positive'
        elif TextBlob(text).sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
    
    def extract_hashtags(self,text):

        regex = "#(\w+)" 
        hashtag_list = re.findall(regex, text)
        return hashtag_list

    def remove_url(self,txt):

        return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())
    
    def remove_url_th(self,txt):
        return " ".join(re.sub("([^\u0E00-\u0E7Fa-zA-Z' ]|^'|'$|''|(\w+:\/\/\S+))", "", txt).split())

    def get_related_tweets(self,key_word,until):

        tweet_keyword = []
        twitter_users = []
        twitter_users_location = []
        tweet_hashtag = []
        tweet_time = []
        tweet_string = [] 
        tweet_countRT = []
        tweet_fav = []
        tweet_sentiment = []
        tweet_polarity = []
        tweet_language = []
        for tweet in tw.Cursor(self._api.search_tweets,
                                q=key_word,
                                tweet_mode="extended",
                                until=until,
                                include_entities=True).items(20):
                                
            if(tweet.lang == 'en'or tweet.lang == 'th'):
                twitter_users.append(tweet.user.screen_name)
                twitter_users_location.append(tweet.user.location)
                tweet_time.append(tweet.created_at)
                tweet_countRT.append(tweet.retweet_count)
                tweet_fav.append(tweet.favorite_count)
                tweet_keyword.append(key_word)
                tweet_hashtag.append(str(self.extract_hashtags(tweet.full_text)))
                tweet_language.append(tweet.lang)
                if tweet.lang == 'en':
                    tweet_string.append(self.remove_url(tweet.full_text))
                    tweet_polarity.append(self.getSentiment(tweet.full_text))
                    tweet_sentiment.append(TextBlob(tweet.full_text).sentiment.polarity)
                elif tweet.lang == 'th':
                    tweet_string.append(self.remove_url_th(tweet.full_text))
                    text = re.sub(r'[%]',' ',tweet.full_text)
                    params = {'text':text}
                    response = requests.get(self._url, headers=self._headers, params=params)
                    try:
                        polarity = str(response.json()['sentiment']['polarity'])
                        sentiment = str(response.json()['sentiment']['score'])
                    except (KeyError):
                        polarity = 'neutral'
                        sentiment = 0
                    tweet_polarity.append(polarity)
                    tweet_sentiment.append(sentiment)

        self.df = pd.DataFrame({'Keyword':tweet_keyword,'User':twitter_users,'Tweet': tweet_string,'Language':tweet_language, 'Time': tweet_time,'User Location':twitter_users_location,
                            'Hashtag':tweet_hashtag,'Polarity':tweet_polarity,'Likes':tweet_fav,'Retweet':tweet_countRT,'Sentiment':tweet_sentiment})

        
        self.df['Time'] = pd.to_datetime(self.df['Time']).dt.strftime('%Y-%m-%d')
        #self.df['Time'] = pd.to_datetime(self.df['Time'])
        folder = "collectkeys"
        path = str(folder+'/'+key_word)
        days = list(set(self.df['Time'].tolist()))
        if key_word not in self.keys:
            
            if not os.path.exists(path):    
                os.mkdir(path)              #create direc for keyword
            for d in days:
                dfff = self.df.loc[self.df['Time'].isin([d])]
                dfff.to_csv(path+'/'+key_word+'_'+d+'.csv',encoding='utf-8',index=False)
            print('save new file comp')
        else:
            print('save old key')
            for d in days:
                dfff = self.df.loc[self.df['Time'].isin([d])]
                csvname = str(path+'/'+key_word+'_'+d+'.csv')
                if csvname in glob.glob(str(str(os.getcwd())+"\\collectkeys\\*.csv")):
                    olddf = pd.read_csv(csvname)
                    newdf = pd.concat([dfff,olddf])
                    newdf.drop_duplicates(keep='last',inplace=True)
                    os.remove(csvname)
                    newdf.to_csv(filename,encoding='utf-8',index=False)
                else:
                    dfff.to_csv(csvname,encoding='utf-8',index=False)
            print('save file comp')
        return self.df

    def savedata(self,keyword,until): #keyword is list

        current_time = datetime.now().strftime("%H:%M:%S")
        print('\nstart saving @',current_time)
        #today = datetime.today()
        #filename = str("tweet_data_"+str(today.day)+str(today.month)+str(today.year)+".csv")

        # if filename not in glob.glob("*.csv"):
        #     self.df = pd.DataFrame(columns=['Keyword','User','Tweet','Language','Time','User Location','Hashtag','Polarity','Likes','Retweet','Sentiment'])
        # else:
        #     self.df = pd.read_csv(filename)

        for kw in keyword:
            self.df = pd.concat([self.df,self.get_related_tweets(kw,until)])

        self.df.drop_duplicates(keep='last',inplace=True)
        self.df.sort_values(by=['Keyword'],inplace=True)
        # if filename in glob.glob("*.csv"):
        #     os.remove(filename)
        # self.df.to_csv(filename,encoding='utf-8',index=False)
        current_time = datetime.now().strftime("%H:%M:%S")
        print('save complete @',current_time)

    def searchkeys(self,keyword,Ans,until):   #keyword's type is list
        #print('keyword',keyword,'\nkeys',self.keys,'\nkeyword in keys?',keyword in self.keys)
        # if "" in keyword:
        #     return self.df
        
        if len(keyword) > 1:            #>1 keyword
            dhave = []
            for key in keyword:
                if key not in self.keys:
                    dhave.append(key)
            print(keyword)
            print(dhave)
            if len(dhave) > 0:          #search new keyword
                
                self.savedata(dhave,until)
                self.keys.extend(dhave)
                return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])
            elif Ans == "real":         #search old keys real time
                self.savedata(keyword,until)
                return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])
            else:
                return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])
        elif keyword[0] in self.keys:   #1 key in old keys
            if Ans == "real":
                self.savedata(keyword,until)
                return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])
            else:
                return self.df.loc[self.df['Keyword']==keyword[0]]
        else:              #1keyword (new)
            if Ans == 'yes':            #new key 1 key
                
                self.savedata(keyword,until)
                self.keys.extend(keyword)
                return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])
            # elif Ans == "real":
            #     self.savedata(keyword,until)
            #     return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])
            else:
                print('You select NO')
                return self.df.loc[self.df['Keyword'].isin(keyword)].sort_values(by=['Keyword'])

In [27]:
dm = DataManager()
dm.newUnion()
dm.df.head()

,Keyword,User,Tweet,Language,Time,User Location,Hashtag,Polarity,Likes,Retweet,Sentiment
0,2d animation,AnimatorsPal,RT AnimationVFXJob Marvel Streaming Production...,en,2022-03-21,Berlin,"['freelanceremotecontractworkfromhome', 'produ...",neutral,0.0,1.0,0.000000
1,2d animation,SirFowlman,RT OleLoken I drew a lady on a vespa then i dr...,en,2022-03-21,Australia,"['2d', '2danimation']",positive,0.0,50.0,0.750000
2,2d animation,cadetcollar,this 100 true lol as an animation student i wa...,en,2022-03-21,she/her | 22,[],positive,5.0,0.0,0.359524
3,2d animation,_Fungai_,RT animationjobs 2D Production Supervisor job ...,en,2022-03-21,NaN,[],neutral,0.0,1.0,0.000000
4,2d animation,peridopp,RT KoalaDingoLabo The koala facing the frogsga...,en,2022-03-21,Scotland,"['gamedev', 'madewithunity', 'unity3d', 'metro...",neutral,0.0,14.0,0.000000


In [28]:
dm.collectwords(dm.df)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,Word,Count
0,animation,70106
1,anime,56370
2,Titan,40059
3,Attack,38897
4,amp,34852
...,...,...
157209,สวมบทบาท,1
157210,929,1
157211,บังหน้า,1
157212,พิทักษ์,1


In [ ]:
tws = Twitter_Scrap()
tws.setdataframe(dm.df)
tws.searchkeys(['spy x family'],'real','2022-4-17')


start saving @ 21:22:59
save old key
save file comp
save complete @ 21:23:01


,Keyword,User,Tweet,Language,Time,User Location,Hashtag,Polarity,Likes,Retweet,Sentiment
19,spy x family,uuzilla,RT museacgth SPY X FAMILY 9 2200,th,2022-03-26,NaN,[],NaN,0.0,202.0,0.0
17,spy x family,arxxct_,RT jamthofficial Spy x Family 9 Nerflix JapanA...,th,2022-03-26,/ ⛓️,['JapanAnimeMovieTha'],NaN,0.0,455.0,0.0
7,spy x family,OwiDkami,RT Crunchyroll SPY x FAMILY arrives on Crunchy...,en,2022-03-26,NaN,[],neutral,0.0,1720.0,0.0
6,spy x family,tob4ias,RT animetvjp Character VisualsSPY x FAMILY Ani...,en,2022-03-26,NaN,[],neutral,0.0,804.0,0.0
5,spy x family,hewkapii,RT jamthofficial Spy x Family 9 Nerflix JapanA...,th,2022-03-26,、มุ่ / #ยกเลิกม.112,['JapanAnimeMovieTha'],NaN,0.0,455.0,0.0
4,spy x family,_assckerman,RT Crunchyroll SPY x FAMILY arrives on Crunchy...,en,2022-03-26,NaN,[],neutral,0.0,1720.0,0.0
3,spy x family,surlysagittari,RT Crunchyroll SPY x FAMILY arrives on Crunchy...,en,2022-03-26,NaN,[],neutral,0.0,1720.0,0.0
2,spy x family,aintapril,RT animetvjp Character VisualsSPY x FAMILY Ani...,en,2022-03-26,capyper land,[],neutral,0.0,804.0,0.0
8,spy x family,kaitogaming1412,AniTrendz Looking forward for spy x family to ...,en,2022-03-26,NaN,[],positive,0.0,0.0,0.5
0,spy x family,yoosunfish,RT Crunchyroll SPY x FAMILY arrives on Crunchy...,en,2022-03-26,she/her | 20,[],neutral,0.0,1720.0,0.0


In [ ]:
folder = "collectkeys"
path = str(folder+'/'+'pixarr')
os.path.exists(path)

False

In [ ]:
dm.collectfile()

In [ ]:
import shutil
path=os.getcwd()
shutil.rmtree(path+'//collectkeys//')

In [ ]:
df1 = dm.getrowwithkeys(['shounen']).sort_values('Keyword')
df1

In [ ]:
dm.setnewDF(df1)
dm.getperiod('2022-03-21','2022-03-25')

In [ ]:
dm.setdefaultDF()

In [ ]:
for word in dm.collectword()[:10]:
    print(word)

In [ ]:
df = pd.DataFrame(dm.collectword()[:100],columns=['Word','Count'])
df

In [ ]:
dm = DataManager()
dm.unionfile(filename)
dm.formatdatetime('Time')
print(dm.df['Time'].min().strftime('%Y/%m/%d'))
print(dm.df['Time'].max().strftime('%Y/%m/%d'))

In [ ]:
keys = ['bl anime','anime comedy','anime romance','ต่างโลก','anime','animation','shounen','pixar',
        'harem','fantasy anime','sport anime','attack on titan','disney animation','animation studio',
        'shounen ai','shoujo','อนิเมะ','2d animation','อนิเมะแนะนำ','japan animation']
len(keys)

In [ ]:
import pandas as pd
import re
import glob
filename = ['tweet_data_2032022.csv','tweet_data_1932022.csv','tweet_data_2132022.csv','tweet_data_2232022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1

df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%Y/%m/%d')
#df['Time'] = pd.to_datetime(df['Time']).dt.strftime('%d %m %Y')
df['Time'] = pd.to_datetime(df['Time'])
df.sort_values(by=['Time','Keyword'],inplace=True)

#time = list(set(df['Time'].tolist()))
mask = (df['Time']>='2022-03-19') & (df['Time']<='2022-03-21')
#mask = (df['Time']>='19-03-2022') & (df['Time']<='21-03-2022')
df.loc[mask]

In [ ]:
keyword = ""
keyword = list(map(lambda x: x.lower(), keyword))
keyword
print(type(None))

In [ ]:
from pythainlp.corpus import thai_stopwords
th_stopwords = list(thai_stopwords())

en_stops = set(stopwords.words('english'))
filename = ['tweet_data_2732022.csv']
i = 0
for file in filename:
    df1 = pd.read_csv(file)
    if i != 0:
        df = pd.concat([df,df1])
        df.drop_duplicates(keep='last',inplace=True)
    else:
        df = df1
        i += 1
        
dff = df.loc[df['Keyword'].isin(['ต่างโลก','อนิเมะ'])]
dff = dff.reset_index()##
word = {}

for index,row in dff.iterrows():##
    if row['Language'] == 'en':
        allwords = row['Tweet'].split()
        for w in allwords: 
            if w not in en_stops:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    elif row['Language'] == 'th':   #
        allwords = word_tokenize(row['Tweet'], engine='newmm')  #
        for w in allwords: 
            if w not in th_stopwords:
                if w in word:
                    word[w] += 1
                else:
                    word[w] = 1
    else:
        pass
del word['RT']
del word[' ']
sortword = sorted(word.items(),key=lambda x:x[1],reverse=True)
for i in sortword[:10]:
    print(i)

FileNotFoundError: [Errno 2] No such file or directory: 'tweet_data_2732022.csv'

In [ ]:
dff = dff.reset_index()
for index,row in dff.iterrows():
    print(row['Tweet'],row['Language'])

In [ ]:
from nltk.corpus import stopwords
en_stops = set(stopwords.words('english'))

all_words = ['There', 'is', 'a', 'tree','near','the','river']
for word in all_words: 
    if word not in en_stops:
        print(word)

In [ ]:
import requests
 
url = "https://api.aiforthai.in.th/ssense"
 
text = 'สาขานี้พนักงานน่ารักให้บริการดี'
 
data = {'text':text}
 
headers = {
    'Apikey': "vIQAf35aRkc7QUbR1fTPvzvtkqtSKAaz"
    }
 
response = requests.post(url, data=data, headers=headers)
 
print(response.json())
response.json()['preprocess']['keyword']

In [ ]:
key = list(set(df['Keyword'].tolist()))

In [ ]:
dm = DataManager()
df = dm.unionfile(filename)
df

In [ ]:
df["Time"] = pd.to_datetime(df["Time"]).dt.strftime('%Y-%m-%d')
keys = list(set(df['Keyword'].tolist()))
folder = "collectkeys"

if not os.path.exists(folder):
    os.mkdir(folder)

for key in keys:
    path = str(folder+'/'+key)
    dff = df.loc[df['Keyword'].isin([key])]
    days = list(set(dff['Time'].tolist()))
    if not os.path.exists(path):
        os.mkdir(path)
    for d in days:
        dfff = dff.loc[dff['Time'].isin([d])]
        dfff.to_csv(path+'/'+key+'_'+d+'.csv',encoding='utf-8',index=False)